In [1]:
import pandas as pd
import numpy as np
import joblib
import os
import json
import warnings
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, f1_score, recall_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from imblearn.over_sampling import BorderlineSMOTE 
from imblearn.pipeline import Pipeline

warnings.filterwarnings('ignore')

# 경로 설정
data_path = '../data/spotify_churn_dataset.csv'
model_save_path = '../models/spotify_churn_model.pkl'
metrics_path = '../data/model_metrics.json'

print("머신러닝 학습을 시작합니다...")
df = pd.read_csv(data_path)
if 'user_id' in df.columns: df = df.drop(columns=['user_id'])

# 피처 엔지니어링
df['ad_burden'] = df['ads_listened_per_week'] / (df['listening_time'] + 1)
df['satisfaction_score'] = df['songs_played_per_day'] * (1 - df['skip_rate'])
df['time_per_song'] = df['listening_time'] / (df['songs_played_per_day'] + 1)

X = df.drop(columns=['is_churned'])
y = df['is_churned']

# 데이터 전처리 및 분할
numerical_cols = ['age', 'listening_time', 'songs_played_per_day', 'skip_rate', 'ads_listened_per_week', 'offline_listening', 'ad_burden', 'satisfaction_score', 'time_per_song']
categorical_cols = ['gender', 'country', 'subscription_type', 'device_type']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_cols)
    ])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 4가지 모델에 대해 경쟁 시작
models_params = {
    "RandomForest": {
        "model": RandomForestClassifier(random_state=42, n_jobs=-1, class_weight='balanced_subsample'),
        "params": {"classifier__n_estimators": [300, 500], "classifier__max_depth": [10, 20]}
    },
    "XGBoost": {
        "model": XGBClassifier(random_state=42, n_jobs=-1, eval_metric='logloss', scale_pos_weight=2.0),
        "params": {"classifier__n_estimators": [500, 1000], "classifier__learning_rate": [0.05]}
    },
    "LightGBM": {
        "model": LGBMClassifier(random_state=42, n_jobs=-1, verbose=-1, scale_pos_weight=2.0),
        "params": {"classifier__n_estimators": [500, 1000], "classifier__learning_rate": [0.05]}
    },
    "CatBoost": {
        "model": CatBoostClassifier(random_state=42, verbose=0, auto_class_weights='SqrtBalanced'),
        "params": {"classifier__iterations": [500, 1000], "classifier__learning_rate": [0.05]}
    }
}

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
bsmote = BorderlineSMOTE(random_state=42)

metrics_data = {}
best_model_name = ""
best_model_score = -1
best_model_pipeline = None

for name, config in models_params.items():
    print(f"[{name}] 모델 최적화 중...")
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('smote', bsmote),
        ('classifier', config['model'])
    ])
    
    search = RandomizedSearchCV(pipeline, config['params'], n_iter=2, scoring='f1', cv=cv, n_jobs=-1, random_state=42)
    search.fit(X_train, y_train)
    
    y_pred = search.best_estimator_.predict(X_test)
    f1 = f1_score(y_test, y_pred)
    
    metrics_data[name] = {
        'Accuracy': float(accuracy_score(y_test, y_pred)),
        'Recall': float(recall_score(y_test, y_pred)),
        'F1-Score': float(f1)
    }
    
    if f1 > best_model_score:
        best_model_score = f1
        best_model_name = name
        best_model_pipeline = search.best_estimator_

# 선정된 최고 모델의 임계값 튜닝
y_proba = best_model_pipeline.predict_proba(X_test)[:, 1]
final_thresh, final_f1 = 0.5, 0
for thresh in np.arange(0.3, 0.7, 0.01):
    f1_t = f1_score(y_test, (y_proba >= thresh).astype(int))
    if f1_t > final_f1:
        final_f1 = f1_t
        final_thresh = thresh

final_pred = (y_proba >= final_thresh).astype(int)
metrics_data[best_model_name].update({
    'Accuracy': float(accuracy_score(y_test, final_pred)),
    'Recall': float(recall_score(y_test, final_pred)),
    'F1-Score': float(final_f1),
    'Best Threshold': float(final_thresh)
})

# 기존 딥러닝 점수가 있다면 유지하면서 저장
if os.path.exists(metrics_path):
    try:
        with open(metrics_path, 'r') as f:
            existing = json.load(f)
            if 'Deep Learning (DNN)' in existing:
                metrics_data['Deep Learning (DNN)'] = existing['Deep Learning (DNN)']
    except: pass

with open(metrics_path, 'w') as f:
    json.dump(metrics_data, f, indent=4)

joblib.dump(best_model_pipeline, model_save_path)
print(f"🏆 최종 선정 모델: {best_model_name} (결과 저장 완료)")

머신러닝 학습을 시작합니다...
[RandomForest] 모델 최적화 중...
[XGBoost] 모델 최적화 중...
[LightGBM] 모델 최적화 중...
[CatBoost] 모델 최적화 중...
🏆 최종 선정 모델: RandomForest (결과 저장 완료)
